This notebook is responsible for visualizing the data from the Github API, which should be provided in a serialized format.

It must be possible to run this notebook using jupyterlite.

If you do not see the newest version of the notebook, you have to "delete" it in your jupyterlite instance, as your local version is overriding the remote version provided by the deployment.

(jupyterlite) We need to manually install dependencies for the front-end, even though they are also defined in the `requirements.txt`.

TODO: Fixed versions to prevent version-drift.

### Troubleshooting:

#### Even after reloading (deep-refreshing) the page, a notebook is not updated

This likely occurred because the backend and frontend of jupyterlite are out of sync.
I am not yet sure why this happens even after a deep-refresh of the page, as this should update the frontend according to the jupyterlite documentation.
As a workaround, delete the cache/cookies for the page and reload it.
Note that you this will reset all notebooks to their versions saved on Github, so download your notebooks if necessary.
You will also have to re-run the notebooks.

#### The `commits.json` cannot be found when loading

Same reasoning & workaround as above.

In [ ]:
%pip install plotly matplotlib ipywidgets

In [ ]:
import json
import datetime
import numpy as np
import plotly
import ipywidgets
from IPython.display import display, clear_output

TODO: We could use the file upload widget instead of requiring the user to already have uploaded the file to jupyterlite.

In [ ]:
# Read the commit data from the 'commits.json'
with open('commits.json') as f:
	commits = json.load(f)

In [ ]:
# Convert all the commit dates to datetime objects for easier handling later on
commit_dates = [datetime.datetime.strptime(commit['author']['date'], '%Y-%m-%dT%H:%M:%SZ') for commit in commits]

This cell handles the number and date ranges of your sprints.
Use the slider to change the number of sprints, and then use the text field and date picker to set a custom name and date range for each sprint.
The number of displayed date pickers will change automatically when changing the number of sprints.
Names and dates for "removed" sprints will be kept, but not displayed.

In [ ]:
# Setting up the number & date ranges of sprints
# All of this needs to be done in one cell as otherwise the DateTime widgets will be sent to the wrong output cell when changing the number of sprints
out = ipywidgets.Output()

num_sprints = ipywidgets.IntSlider(min=1, max=10, step=1, description='No. of sprints')
out.append_display_data(num_sprints)
out.append_stdout('Set a name and the start and end dates for each sprint below:')

sprint_names = []
start_dates = []
end_dates = []

# Initialize the widgets for the first sprint
sprint_names.append(ipywidgets.Text(value=f'Sprint 1', placeholder='Set the name for this Sprint', description='Sprint Name:'))
start_dates.append(ipywidgets.DatePicker(description='Start Date'))
end_dates.append(ipywidgets.DatePicker(description='End Date'))
out.append_display_data(sprint_names[0])
out.append_display_data(start_dates[0])
out.append_display_data(end_dates[0])

# Automatically add/remove the widgets based on the number of sprints
def update_sprints(change):
	if change['type'] == 'change' and change['name'] == 'value':
		if change['new'] > change['old']:
			# Only add a new widget if we do not have one with this number already
			if len(sprint_names) < change['new']:
				sprint_names.append(ipywidgets.Text(value=f'Sprint {change["new"]}', placeholder='Set the name for this Sprint', description='Sprint Name:'))
				start_dates.append(ipywidgets.DatePicker(description='Start Date'))
				end_dates.append(ipywidgets.DatePicker(description='End Date'))
			# Display the necessary widgets for the new sprint
			out.append_display_data(sprint_names[change["new"] - 1])
			out.append_display_data(start_dates[change["new"] - 1])
			out.append_display_data(end_dates[change["new"] - 1])
		elif change['new'] < change['old']:
			# Remove the widgets for the removed sprint (workaround, as clear_output() will not work here, as we are using append_display_data() instead of the 'with out:' syntax)
			out.outputs = out.outputs[:-3]

num_sprints.observe(update_sprints, names='value')

display(out)

This cell contains the logic for the commit heatmap.
For a given time interval, the number of commits per day and hour of the day is displayed in a heatmap.

In [ ]:
# Plot a heatmap for the number of commits per day and hour of the week
def plot_commit_heatmap(commit_dates, start_date=None, end_date=None):
	# Convert date objects to datetime objects
	if start_date is not None:
		start_date = datetime.datetime.combine(start_date, datetime.datetime.min.time())
	if end_date is not None:
		end_date = datetime.datetime.combine(end_date, datetime.datetime.max.time())
	# Filter the dates to only include those between the start and end dates
	# The dates are already sorted, so we can just filter the beginning and end
	if start_date is not None:
		commit_dates = [date for date in commit_dates if date >= start_date]
	if end_date is not None:
		commit_dates = [date for date in commit_dates if date <= end_date]
	
	# Get the day and hour of each commit
	days = [date.weekday() for date in commit_dates]
	y_vals = [date.hour for date in commit_dates]
	
	# Create a 2D histogram of the day and hour
	heatmap, xedges, yedges = np.histogram2d(days, y_vals, bins=(7, 24))
	
	# Create a plotly figure
	fig = plotly.graph_objs.FigureWidget()
	
	# Add axis labels
	fig.update_layout(
		xaxis=dict(
			title='Hour of the day',
			tickmode='array',
			tickvals=np.arange(0, 24, 2),
			ticktext=np.arange(0, 24, 2)
		),
		yaxis=dict(
			title='Day of the week',
			tickmode='array',
			tickvals=np.arange(0, 7),
			ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
		)
	)
	
	# Add a heatmap trace
	fig.add_trace(plotly.graph_objs.Heatmap(
		z=heatmap,
		x=xedges,
		y=yedges,
		colorscale='oranges',
		colorbar=dict(
			title='Commits'
		),
		hoverinfo='z'
	))
	
	return fig

This cell handles the display of the heatmap for the given sprints.
Only one heatmap is displayed at a time, the sprint for which this is done can be selected using the dropdown menu.
If you change the date ranges of the sprints, you *do not* have to re-run this cell.
However, you do need to re-run the cell if you have changed the number of sprints, as this is not currently handled automatically.

In [ ]:
# Add a widget to select the sprint to display
sprint_selector = ipywidgets.Dropdown(
	options=[(sprint_names[i].value, i) for i in range(num_sprints.value)],
	description='Sprint:',
	disabled=False,
)
display(sprint_selector)

fig = plot_commit_heatmap(commit_dates, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value)
fig.update_layout(title=f'Commit timings during {sprint_names[sprint_selector.value].value} ({start_dates[sprint_selector.value].value} - {end_dates[sprint_selector.value].value})')
display(fig)

def update_sprint(change):
	fig.data[0].z = plot_commit_heatmap(commit_dates, start_dates[change['new']].value, end_dates[change['new']].value).data[0].z
	fig.update_layout(title=f'Commit timings during {sprint_names[change["new"]].value} ({start_dates[change["new"]].value} - {end_dates[change["new"]].value})')

sprint_selector.observe(update_sprint, names='value')